In [573]:
import pandas as pd
import numpy as np

In [574]:
# Import all Excel Files

order=pd.read_excel("Company X - Order Report.xlsx")
pincode=pd.read_excel("Company X - Pincode Zones.xlsx")
sku=pd.read_excel("Company X - SKU Master.xlsx")
invoice=pd.read_excel("Courier Company - Invoice.xlsx")
rate=pd.read_excel("Courier Company - Rates.xlsx")

In [575]:
# Check duplicate

print('pincode',pincode.duplicated().sum())
print('sku',sku.duplicated().sum())
print('invoice',invoice.duplicated().sum())

pincode 16
sku 1
invoice 0


In [576]:
# Delete duplicate values

pincode.drop_duplicates(inplace=True)
pincode.reset_index(drop=True,inplace=True)

sku.drop_duplicates(inplace=True)
sku.reset_index(drop=True,inplace=True)

In [577]:
# Capitalizing the Zone

pincode["Zone"]=pincode["Zone"].str.capitalize()
invoice["Zone"]=invoice["Zone"].str.capitalize()

In [578]:
# Renaming Coulmn
order.rename(columns={'ExternOrderNo': 'Order ID'}, inplace=True)

In [579]:
# Creating new calculation table

calculation= pd.DataFrame(invoice["Order ID"])
calculation["AWB Cade"] = invoice["AWB Code"]
calculation["Total Weight as per Courier Company (KG)"] = invoice["Charged Weight"]
calculation["Delivery Zone charged by courier company"] = invoice["Zone"]
calculation["Invoice Amount (Rs.)"] = invoice["Billing Amount (Rs.)"]
calculation["Customer Pincode"] = invoice["Customer Pincode"]
calculation["Type of Shipment"] = invoice["Type of Shipment"]

In [580]:
# merge sku with corrseponding weight
order=pd.merge(order,sku, on ='SKU', how ='left')

# Calculating total weight by quantity
order["Total Weight as per X (KG)"]=order["Order Qty"]*order["Weight (g)"]/1000

# deleting the extra row
order.drop('Weight (g)',axis=1,inplace=True)

In [581]:
# Calculating total weight groug by Order ID
Order_weight=order.groupby('Order ID')['Total Weight as per X (KG)'].sum()

# Converting to datafrme
Order_weight=Order_weight.reset_index()

# Merge with calculation table
calculation=pd.merge(calculation,Order_weight, on ='Order ID', how ='left')

In [582]:
# Merge pincode with zone
calculation=pd.merge(calculation,pincode, on ='Customer Pincode', how ='left')

# Deleting extra column
calculation.drop('Customer Pincode',axis=1,inplace=True)
calculation.drop('Warehouse Pincode',axis=1,inplace=True)

In [583]:
# Merge rate table
calculation=pd.merge(calculation,rate, on ='Zone', how ='left')

# calculating weight slab as per X
calculation['Aditional slab']= (calculation['Total Weight as per X (KG)']/calculation['Weight Slabs']).apply(np.ceil)-1
calculation['Weight slab as per X (KG)']= (calculation['Aditional slab']+1) * calculation['Weight Slabs']

In [584]:
# Function to calculate Expected charges as per X

def total_cost(row):
    if row['Type of Shipment'] == "Forward charges":
        return row['Forward Fixed Charge']+row['Aditional slab']*row['Forward Additional Weight Slab Charge']
    else:
        return row['Forward Fixed Charge']+row['Aditional slab']*row['Forward Additional Weight Slab Charge'] + row['RTO Fixed Charge']+row['Aditional slab']*row['RTO Additional Weight Slab Charge']
    
    
calculation['Expected Charge as per X (Rs.)'] = calculation.apply(total_cost,axis=1)

In [585]:
# Rename Column 
calculation.rename(columns={'Zone': 'Delivery Zone as per X'}, inplace=True)

# Deleting extra column
calculation.drop('Weight Slabs',axis=1,inplace=True)
calculation.drop('Forward Fixed Charge',axis=1,inplace=True)
calculation.drop('Forward Additional Weight Slab Charge',axis=1,inplace=True)
calculation.drop('RTO Fixed Charge',axis=1,inplace=True)
calculation.drop('RTO Additional Weight Slab Charge',axis=1,inplace=True)
calculation.drop('Aditional slab',axis=1,inplace=True)

In [586]:
# Rename Column to match the column name
rate.rename(columns={'Zone': 'Delivery Zone charged by courier company'}, inplace=True)

calculation=pd.merge(calculation,rate, on ='Delivery Zone charged by courier company', how ='left')

# calculating weight slab charged by courier company
calculation['Aditional slab']= (calculation['Total Weight as per Courier Company (KG)']/calculation['Weight Slabs']).apply(np.ceil)-1
calculation['Weight slab charged by Courier Company (KG)']= (calculation['Aditional slab']+1) * calculation['Weight Slabs']

In [587]:
# Function to calculate Charges Billed by Courier Company

def total_cost(row):
    if row['Type of Shipment'] == "Forward charges":
        return row['Forward Fixed Charge']+row['Aditional slab']*row['Forward Additional Weight Slab Charge']
    else:
        return row['Forward Fixed Charge']+row['Aditional slab']*row['Forward Additional Weight Slab Charge'] + row['RTO Fixed Charge']+row['Aditional slab']*row['RTO Additional Weight Slab Charge']
    

calculation['Charges Billed by Courier Company (Rs.)'] = calculation.apply(total_cost,axis=1)

In [588]:
# Deleting extra column
calculation.drop('Weight Slabs',axis=1,inplace=True)
calculation.drop('Forward Fixed Charge',axis=1,inplace=True)
calculation.drop('Forward Additional Weight Slab Charge',axis=1,inplace=True)
calculation.drop('RTO Fixed Charge',axis=1,inplace=True)
calculation.drop('RTO Additional Weight Slab Charge',axis=1,inplace=True)
calculation.drop('Aditional slab',axis=1,inplace=True)
calculation.drop('Type of Shipment',axis=1,inplace=True)

In [589]:
# calculating Difference Between Expected Charges and Billed Charges

calculation['Difference Between Expected Charges and Billed Charges (Rs.)']= calculation['Expected Charge as per X (Rs.)']-calculation['Charges Billed by Courier Company (Rs.)']

In [590]:
# Creating final calculation table

calculation_table= calculation[['Order ID', 'AWB Cade','Total Weight as per X (KG)', 'Weight slab as per X (KG)', 
                                'Total Weight as per Courier Company (KG)','Weight slab charged by Courier Company (KG)', 
                                'Delivery Zone as per X', 'Delivery Zone charged by courier company', 
                                'Expected Charge as per X (Rs.)', 'Charges Billed by Courier Company (Rs.)',
                                'Difference Between Expected Charges and Billed Charges (Rs.)']]

In [591]:
# Count values for summary table

charged_correctly_count = (calculation['Difference Between Expected Charges and Billed Charges (Rs.)'] == 0).sum()
over_charged_count = (calculation['Difference Between Expected Charges and Billed Charges (Rs.)'] < 0).sum()
under_charged_count = (calculation['Difference Between Expected Charges and Billed Charges (Rs.)'] > 0).sum()


# sum values for summary table

total_invoice_amount = calculation[calculation['Difference Between Expected Charges and Billed Charges (Rs.)']==0]['Invoice Amount (Rs.)'].sum()
total_overcharging_amount = calculation[calculation['Difference Between Expected Charges and Billed Charges (Rs.)']<0]['Difference Between Expected Charges and Billed Charges (Rs.)'].sum()
total_undercharging_amount = calculation[calculation['Difference Between Expected Charges and Billed Charges (Rs.)']>0]['Difference Between Expected Charges and Billed Charges (Rs.)'].sum()

In [592]:
# Creating summary table

data= { 'Count' :[charged_correctly_count,over_charged_count,under_charged_count],
       'Amount (Rs.)' : [total_invoice_amount,total_overcharging_amount,total_undercharging_amount]}

index=['Total orders where X has been correctly charged',
       'Total Orders where X has been overcharged',
       'Total Orders where X has been undercharged']

summary_table= pd.DataFrame(data,index=index)

In [594]:
summary_table

,Count,Amount (Rs.)
Total orders where X has been correctly charged,48,5955.6
Total Orders where X has been overcharged,49,-1274.9
Total Orders where X has been undercharged,27,580.8


In [598]:
# export to excel file

file_name=pd.ExcelWriter('Cointab_assignment_result.xlsx')

summary_table.to_excel(file_name, sheet_name='Summary')
calculation_table.to_excel(file_name, sheet_name='Calculation',index=False)

file_name.save()

In [599]:
import pandas as pd

# Example DataFrames
data1 = {'Column1': [1, 2, 3]}
data2 = {'ColumnA': ['A', 'B', 'C']}
df1 = pd.DataFrame(data1)
df2 = pd.DataFrame(data2)

# Create an ExcelWriter object
excel_writer = pd.ExcelWriter('output.xlsx')

# Write each DataFrame to a different sheet
df1.to_excel(excel_writer, sheet_name='Sheet1', index=False)
df2.to_excel(excel_writer, sheet_name='Sheet2', index=False)

# Save and close the ExcelWriter
excel_writer.save()
